In [ ]:
import ee
import numpy as np
import pandas as pd
import os

#service_account = 'encaramelo@alien-fold-235112.iam.gserviceaccount.com'
#credentials = ee.ServiceAccountCredentials(service_account, '/content/drive/My Drive/Colab Notebooks/alien-fold-235112-e3ab4bc3ddd0.json')
#ee.Initialize(credentials)

In [ ]:
## Initialise system
#import ee
#import numpy as np
#import pandas as pd
#import os

## Trigger the authentication flow.
ee.Authenticate()

## Initialise the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=DEfJtQGBV7noFoWkTF6td-TuHavveX2C_EAezeSq3R4&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# aux functions for time
import datetime 
import time

def get_timestamp(in_date):
    utc_naive  = in_date.replace(tzinfo=None) - in_date.utcoffset()
    timestamp = (utc_naive - datetime.datetime(1970, 1, 1)).total_seconds()
    return (timestamp)


def get_timestamp_utc(in_date):
    utc_naive  = in_date.replace(tzinfo=None)
    timestamp = (utc_naive - datetime.datetime(1970, 1, 1)).total_seconds()
    return int(timestamp)

In [ ]:
def fc2df(fc):
    # Convert a FeatureCollection into a pandas DataFrame
    features = fc.getInfo()['features']
    dictarr = []
    for f in features:
        attr = f['properties']
        dictarr.append(attr)

    df = pd.DataFrame(dictarr)
    return df

In [ ]:
def get_image_n_box(latitude, longitude, start_time, end_time):
  """ Returns the clippled image collection and the used box """

  point = ee.Geometry.Point([longitude,latitude]);
  
  S2 = ee.ImageCollection("COPERNICUS/S2_SR");
  # Filtering
  S2TemporalFiltered = S2.filterDate(start_time,end_time);
  S2SpatialFiltered = S2TemporalFiltered.filterBounds(point);
  imagenes = S2SpatialFiltered.size()

  # Box coordinates to reduce collection,
  x1 = longitude - 0.00058;
  x2 = longitude + 0.00058;
  y1 = latitude - 0.000437;
  y2 = latitude + 0.000437;
  box = ee.Geometry.Rectangle(x1,y1,x2,y2);




  # Cutting collection to box
  S2Clipped = S2SpatialFiltered.map(
  lambda img : img.clip(box)
  );

  # Mean values
  eBox = ee.Feature(box);
  return S2Clipped, eBox

In [ ]:
def get_dataframes(S2Clipped, eBox):
  """ Returns Bandas and Properties dataframe """

  mbox_image = lambda image : ee.Feature(None, image.reduceRegion(reducer=ee.Reducer.mean(), geometry=eBox.geometry())) 

  

  get_properties = lambda image : ee.Feature(None,ee.Dictionary({
    "system_index":image.get('system:index'),
    "MEAN_SOLAR_AZIMUTH_ANGLE":image.get('MEAN_SOLAR_AZIMUTH_ANGLE'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B1":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B1'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B2":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B2'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B3":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B3'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B4":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B4'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B5":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B5'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B6":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B6'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B7":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B7'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B8":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B8'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B9":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B9'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B10":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B10'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B11":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B11'),
    "MEAN_INCIDENCE_AZIMUTH_ANGLE_B12":image.get('MEAN_INCIDENCE_AZIMUTH_ANGLE_B12'),
    "SOLAR_IRRADIANCE_B1":image.get('SOLAR_IRRADIANCE_B1'),
    "SOLAR_IRRADIANCE_B2":image.get('SOLAR_IRRADIANCE_B2'),
    "SOLAR_IRRADIANCE_B3":image.get('SOLAR_IRRADIANCE_B3'),
    "SOLAR_IRRADIANCE_B4":image.get('SOLAR_IRRADIANCE_B4'),
    "SOLAR_IRRADIANCE_B5":image.get('SOLAR_IRRADIANCE_B5'),
    "SOLAR_IRRADIANCE_B6":image.get('SOLAR_IRRADIANCE_B6'),
    "SOLAR_IRRADIANCE_B7":image.get('SOLAR_IRRADIANCE_B7'),
    "SOLAR_IRRADIANCE_B8":image.get('SOLAR_IRRADIANCE_B8'),
    "SOLAR_IRRADIANCE_B8A":image.get('SOLAR_IRRADIANCE_B8A'),
    "SOLAR_IRRADIANCE_B9":image.get('SOLAR_IRRADIANCE_B9'),
    "SOLAR_IRRADIANCE_B10":image.get('SOLAR_IRRADIANCE_B10'),
    "SOLAR_IRRADIANCE_B11":image.get('SOLAR_IRRADIANCE_B11'),
    "SOLAR_IRRADIANCE_B12":image.get('SOLAR_IRRADIANCE_B12'),
    "CLOUDY_PIXEL_PERCENTAGE":image.get('CLOUDY_PIXEL_PERCENTAGE'),
    "CLOUD_COVERAGE_ASSESSMENT":image.get('CLOUD_COVERAGE_ASSESSMENT'),
    "REFLECTANCE_CONVERSION_CORRECTION":image.get('REFLECTANCE_CONVERSION_CORRECTION'),
    "MEAN_SOLAR_ZENITH_ANGLE":image.get('MEAN_SOLAR_ZENITH_ANGLE '),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B1":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B1'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B2":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B2'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B3":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B3'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B4":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B4'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B5":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B5'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B6":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B6'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B7":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B7'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B8":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B8'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B8A":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B8A'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B9":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B9'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B10":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B10'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B11":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B11'),
    "MEAN_INCIDENCE_ZENITH_ANGLE_B12":image.get('MEAN_INCIDENCE_ZENITH_ANGLE_B12'),
        
    "DATETIME":image.get('system:time_start')
    # Date objects transferred to the client are milliseconds since UNIX epoch;
    # convert to human readable date with ee.Date.format().
    
    }))  



  df_bandas = fc2df(S2Clipped.map(mbox_image))
  df_propiedades = fc2df(S2Clipped.map(get_properties))
  return df_bandas, df_propiedades

In [ ]:
image.get

NameError: ignored

In [ ]:
# Just specifying start date (end time 1 day after)
# Changed to specify start and end (not only 1 day)

def get_satellite_dataframe_row(latitude, longitude, start_time,end_time):
    #start_time = date.strftime("%Y-%m-%d")
    #end_time = (date + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    
    S2Clipped, eBox = get_image_n_box(latitude, longitude, start_time, end_time)
    df_bandas, df_propiedades = get_dataframes(S2Clipped, eBox)
    
    merged = df_bandas.join(df_propiedades)
    
    if not merged.empty:
        merged.sort_values(by=['system_index'],inplace=True)
        merged=merged.tail(1) # get last
    return merged
  

In [ ]:
# get measurement dataframe (only csv for now)
def load_n_process_measurements_file(file_path):
    """ Load'n'process measurement csv and returns dataframe with rows that fall within an image. 
    Note: limited to one image per day"""
    print("Opening measurement csv file '{}'".format(file_path))
    df_measurements = pd.read_csv(file_path, header=0)
    df_measurements['DATE'] = [datetime.datetime.strptime(i[0:19], '%Y-%m-%d  %H:%M:%S').date() for i in df_measurements['DATETIME']]
    df_out = pd.DataFrame(columns=df_measurements.columns.values)
    for measurement_date in df_measurements.drop_duplicates('DATE', keep='first')['DATE']:
        df_at_measurement_date = df_measurements.loc[df_measurements['DATE'] == measurement_date]
        sat_timestamps = ee_get_picture_date(measurement_date,
                                             df_at_measurement_date["LATITUDE"].min(),
                                             df_at_measurement_date["LATITUDE"].max(),
                                             df_at_measurement_date["LONGITUDE"].min(),
                                             df_at_measurement_date["LONGITUDE"].max() )
        df_temp = pd.DataFrame()
        for sat_timestamp in sat_timestamps:
            df_dif = df_at_measurement_date
            df_dif['TIMESTAMP_DIFF'] = (df_at_measurement_date['TIMESTAMP'] - sat_timestamp).abs()
            min_dif = df_dif['TIMESTAMP_DIFF'].min()
            if min_dif < 3600:
                df_temp = pd.concat([df_temp, df_dif[df_dif['TIMESTAMP_DIFF'] == min_dif]], axis =0)
        if not df_temp.empty:        
            df_temp.sort_values(by=['TIMESTAMP_DIFF'],inplace=True)
            df_out = pd.concat([df_out, df_temp.head(1)], axis =0)
    return df_out


In [ ]:
def timestamp_from_system_index(system_index):
    year = int(system_index[0:4])
    month = int(system_index[4:6])
    day = int(system_index[6:8])
    hour = int(system_index[9:11])
    minutes = int(system_index[11:13])
    return get_timestamp_utc(datetime.datetime(year, month, day, hour, minutes))


S2 = ee.ImageCollection("COPERNICUS/S2");
def ee_get_picture_date(date, min_lat, max_lat, min_lon, max_lon):
  box = ee.Geometry.Rectangle(min_lon,min_lat,max_lon,max_lat);
  start_time = date.strftime("%Y-%m-%d")
  end_time = (date + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
  S2TemporalFiltered = S2.filterDate(start_time, end_time);
  S2SpatialFiltered = S2TemporalFiltered.filterBounds(box);
  imagenes = S2SpatialFiltered.size()


  get_index = lambda image : ee.Feature(None,ee.Dictionary({
    "system_index":image.get('system:index')}))
  fc = S2SpatialFiltered.map(get_index)
  timestamps = []
  for f in fc.getInfo()['features']:
    attr = f['properties']['system_index']
    if attr is not None:
        timestamps.append(timestamp_from_system_index(attr))
  return timestamps


NameError: ignored

In [ ]:
def process_file(input_file, output_file):
  df_points = load_n_process_measurements_file(input_file )
  df_output = pd.DataFrame()
  if not df_points.empty:
    for index, measure_row in df_points.iterrows():
        try:
          sat_row = get_satellite_dataframe_row(latitude=measure_row['LATITUDE'], longitude=measure_row['LONGITUDE'],date=measure_row['DATE'])
        except:
          print("get_satellite_dataframe_row failed, trying again")
          sat_row = get_satellite_dataframe_row(latitude=measure_row['LATITUDE'], longitude=measure_row['LONGITUDE'],date=measure_row['DATE'])
        if not sat_row.empty:
          merged_row = dict()
          merged_row.update(measure_row)
          merged_row.update(sat_row)
          df_merged_row = pd.DataFrame.from_dict(merged_row)
          df_output = pd.concat([df_output, df_merged_row], axis=0)   
  if df_output.empty:
    print("Warning: No valid points found in '{}".format(input_file))
  else:
    df_output['TIME_DIFF'] = df_output.apply(lambda row: abs(int(timestamp_from_system_index(row['system_index']) - row['TIMESTAMP'])), axis=1)
    df_output.sort_values(by=['TIME_DIFF'],inplace=True)
    df_output.drop_duplicates('system_index', keep='first',inplace=True)

  print("*** Complete. Saving file... '{}' ".format(output_file))
  df_output.to_csv(os.path.join(output_dir,output_file), index=False)

In [ ]:
measurements_dir = "/content/drive/My Drive/Colab Notebooks"
output_dir = "/content/drive/My Drive/Colab Notebooks"

j = 0
for measurements_file_csv in os.listdir(measurements_dir):
    j = j+1
    print(j)
    output_file = "sat_with_" + measurements_file_csv
    if not os.path.isfile(os.path.join(output_dir,output_file)):
        print("~~ Processing file '{}' ~~".format(output_file))
        input_file = os.path.join(measurements_dir, measurements_file_csv)
        #process_file(input_file, output_file)
        
        try:
            process_file(input_file, output_file)
      
        except Exception as e:
            try:
                print("! Processing file failed, trying again")
                print(e)
                process_file(input_file, output_file)
            except Exception as e1:
                print("! Processing file failed again, giving up...")
                print(e1)
    else:
        print("~~ Skipping, file '{}' already exists.".format(output_file))

! Processing file failed, trying again
'DATETIME'
Opening measurement csv file '/content/drive/My Drive/Colab Notebooks/DOWSING_20190514.csv'
! Processing file failed again, giving up...
'DATETIME'


In [ ]:
image = 

print('All metadata:')
pprint.pprint(image.getInfo())


In [ ]:
## Downloading / clipping the satellite data


# Using the S2 image box clipping at Dowsing bay Coordinates
from datetime import datetime

latitude = 51.998167
longitude = 2.079333
start_time = "2015-06-01T17:30:00"
end_time = "2021-05-14T00:00:00"

date = datetime.strptime(start_time, "%Y-%m-%dT%H:%M:%S")
date = date.strftime('%m-%d-%Y')
date =datetime.strptime(date,'%m-%d-%Y')


# Considering date is in dd/mm/yyyy format
#start_time = datetime.strptime(start_time, "%d/%m/%YT%H:%M:%S")
#end_time = datetime.strptime(end_time, "%d/%m/%YT%H:%M:%S")


# Taking the clipped image from the filtered point and date/time
S2Clipped,eBox = get_image_n_box(latitude, longitude, start_time, end_time)
S2Clipped

# # Getting dataframes out

df_bands, df_properties = get_dataframes(S2Clipped, eBox)

# # Combining the data frames of Bands and Properties! 
df_total = pd.concat([df_bands, df_properties],axis=1)
df_total

# add in latitude / longitude data. 
# for the dataframe add into the df_total 
# add column lat/long 


#merged = get_satellite_dataframe_row(latitude, longitude, start_time,end_time)
(eBox.area()).getInfo() #tells the area of eBox area (get info pulls it out)


In [ ]:
df_properties.iloc[0,2]
date = ee.Date(int(df_properties.iloc[0,2])).format().getInfo()
date



'2017-03-30T11:03:34'

In [ ]:
from google.colab import files # importing to be able to download to local file

df_total.to_csv('dftotal.csv') # converting dataframe to csv and then downloading 
files.download('dftotal.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Matching process with in-situ Dowsing bay


In [ ]:
satDows = df_properties

lat = 53.53133
lon = 1.053167

lat = [lat]*499
lon = [lon]*499
lat = pd.Series(lat)     #making series for datafram
lat = lat.rename("LATITUDE") #naming series

lon = pd.Series(lon)
lon = lon.rename("LONGITUDE")

satDows.insert(3,"LATITUDE",lat,True) #(adds cluster at column 13)
satDows.insert(3,"LONGITUDE",lon,True) #(adds cluster at column 13)

satDows.iloc[0,2]

1490871814000

In [ ]:
datetime = []

for i in range(0,satDows.shape[0]):
    a = satDows.iloc[i,2]
    
    date_i = ee.Date(int(a)).format().getInfo() # making timestamp into readable date (int->format-> get info)
    datetime.append(date_i)


datetime = pd.Series(datetime)
datetime = datetime.rename("TIMEDATE")

satDows.insert(3,"TIMEDATE",datetime,True) #(adds new date/time at column 3)

In [ ]:
satDows

,CLOUDY_PIXEL_PERCENTAGE,CLOUD_COVERAGE_ASSESSMENT,DATETIME,TIMEDATE,LONGITUDE,LATITUDE,LONGITUDE,LATITUDE,TIMEDATE,LONGITUDE,LATITUDE,MEAN_INCIDENCE_AZIMUTH_ANGLE_B1,MEAN_INCIDENCE_AZIMUTH_ANGLE_B10,MEAN_INCIDENCE_AZIMUTH_ANGLE_B11,MEAN_INCIDENCE_AZIMUTH_ANGLE_B12,MEAN_INCIDENCE_AZIMUTH_ANGLE_B2,MEAN_INCIDENCE_AZIMUTH_ANGLE_B3,MEAN_INCIDENCE_AZIMUTH_ANGLE_B4,MEAN_INCIDENCE_AZIMUTH_ANGLE_B5,MEAN_INCIDENCE_AZIMUTH_ANGLE_B6,MEAN_INCIDENCE_AZIMUTH_ANGLE_B7,MEAN_INCIDENCE_AZIMUTH_ANGLE_B8,MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A,MEAN_INCIDENCE_AZIMUTH_ANGLE_B9,MEAN_INCIDENCE_ZENITH_ANGLE_B1,MEAN_INCIDENCE_ZENITH_ANGLE_B10,MEAN_INCIDENCE_ZENITH_ANGLE_B11,MEAN_INCIDENCE_ZENITH_ANGLE_B12,MEAN_INCIDENCE_ZENITH_ANGLE_B2,MEAN_INCIDENCE_ZENITH_ANGLE_B3,MEAN_INCIDENCE_ZENITH_ANGLE_B4,MEAN_INCIDENCE_ZENITH_ANGLE_B5,MEAN_INCIDENCE_ZENITH_ANGLE_B6,MEAN_INCIDENCE_ZENITH_ANGLE_B7,MEAN_INCIDENCE_ZENITH_ANGLE_B8,MEAN_INCIDENCE_ZENITH_ANGLE_B8A,MEAN_INCIDENCE_ZENITH_ANGLE_B9,MEAN_SOLAR_AZIMUTH_ANGLE,MEAN_SOLAR_ZENITH_ANGLE,REFLECTANCE_CONVERSION_CORRECTION,SOLAR_IRRADIANCE_B1,SOLAR_IRRADIANCE_B10,SOLAR_IRRADIANCE_B11,SOLAR_IRRADIANCE_B12,SOLAR_IRRADIANCE_B2,SOLAR_IRRADIANCE_B3,SOLAR_IRRADIANCE_B4,SOLAR_IRRADIANCE_B5,SOLAR_IRRADIANCE_B6,SOLAR_IRRADIANCE_B7,SOLAR_IRRADIANCE_B8,SOLAR_IRRADIANCE_B8A,SOLAR_IRRADIANCE_B9,system_index
0,88.229800,88.229800,1490871814000,2017-03-30T11:03:34,1.053167,53.53133,1.053167,53.53133,1970-01-01T00:00:00,1.053167,53.53133,106.380731,105.976926,106.175346,106.375165,105.642220,105.805027,105.951912,106.046104,106.123037,106.210643,105.732293,106.273706,106.456698,8.573767,8.424309,8.476681,8.539689,8.400771,8.418980,8.453946,8.479029,8.501709,8.526834,8.411418,8.546440,8.606798,162.291249,None,1.004931,1913.57,367.15,245.59,85.25,1941.63,1822.61,1512.79,1425.56,1288.32,1163.19,1036.39,955.19,813.04,20170330T105621_20170330T110334_T30UYE
1,89.959000,89.959000,1490871814000,2017-03-30T11:03:34,1.053167,53.53133,1.053167,53.53133,1970-01-01T00:00:00,1.053167,53.53133,106.913819,106.242275,106.566141,106.874642,105.730717,106.003116,106.216904,106.349785,106.467552,106.624785,105.859113,106.762252,107.034908,8.757399,8.622553,8.663812,8.725107,8.574662,8.599756,8.632874,8.652601,8.674689,8.701019,8.585040,8.730511,8.789543,162.240618,None,1.004931,1913.57,367.15,245.59,85.25,1941.63,1822.61,1512.79,1425.56,1288.32,1163.19,1036.39,955.19,813.04,20170330T105621_20170330T110334_T31UCV
2,0.000000,0.000000,1491735929000,2017-04-09T11:05:29,1.053167,53.53133,1.053167,53.53133,1970-01-01T00:00:00,1.053167,53.53133,106.297873,105.952827,106.160175,106.319570,105.610829,105.758119,105.908020,105.974910,106.043884,106.123677,105.679140,106.192661,106.368127,8.569965,8.421031,8.464587,8.522300,8.388649,8.406974,8.440906,8.462308,8.485059,8.510261,8.393751,8.537805,8.603053,162.326054,None,0.999176,1913.57,367.15,245.59,85.25,1941.63,1822.61,1512.79,1425.56,1288.32,1163.19,1036.39,955.19,813.04,20170409T105651_20170409T110529_T30UYE
3,0.000000,0.000000,1491735929000,2017-04-09T11:05:29,1.053167,53.53133,1.053167,53.53133,1970-01-01T00:00:00,1.053167,53.53133,106.874162,106.199665,106.510127,106.779327,105.714487,105.950691,106.189800,106.344024,106.461976,106.589848,105.839967,106.706434,106.992951,8.754571,8.607224,8.648667,8.711750,8.571689,8.598126,8.629926,8.651515,8.673624,8.698130,8.582075,8.724904,8.786750,162.273672,None,0.999176,1913.57,367.15,245.59,85.25,1941.63,1822.61,1512.79,1425.56,1288.32,1163.19,1036.39,955.19,813.04,20170409T105651_20170409T110529_T31UCV
4,82.997800,82.997800,1492599961000,2017-04-19T11:06:01,1.053167,53.53133,1.053167,53.53133,1970-01-01T00:00:00,1.053167,53.53133,106.215409,105.872939,106.027163,106.191680,105.572758,105.714744,105.857413,105.917394,105.979592,106.052734,105.643658,106.115139,106.279003,8.540157,8.399039,8.443751,8.507113,8.361711,8.385681,8.417856,8.441755,8.464579,8.489856,8.372421,8.517486,8.573371,162.252394,None,0.993495,1913.57,367.15,245.59,85.25,1941.63,1822.61,1512.79,1425.56

In [ ]:
import pandas as pd


insituDows = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DOWSING_20190514.csv')

In [ ]:
insituDows

,Unnamed: 0,DateTime,longitude,latitude,depth,value,units,parameter,stationid
0,0,10/01/2009 14:00,1.053167,53.531333,1,5.96,°C,TEMP,DOWSING
1,1,10/01/2009 14:00,1.053167,53.531333,1,34.62,PSS78,SAL,DOWSING
2,2,10/01/2009 14:00,1.053167,53.531333,1,2.40,mg/l,SUSPLD,DOWSING
3,3,10/01/2009 14:00,1.053167,53.531333,1,5.89,°C,TEMP,DOWSING
4,4,10/01/2009 14:00,1.053167,53.531333,1,2.11,FTU,FTU,DOWSING
...,...,...,...,...,...,...,...,...,...
844505,844505,14/05/2019 10:00,1.057033,53.529850,1,10.18,°C,TEMP,DOWSING
844506,844506,14/05/2019 10:00,1.057033,53.529850,1,2.94,FTU,FTU,DOWSING
844507,844507,14/05/2019 10:00,1.057033,53.529850,1,5.40,mg/l,SUSPLD,DOWSING
844508,844508,14/05/2019 10:00,1.057033,53.529850,1,34.44,PSS78,SAL,DOWSING


In [ ]:
#taking closest time_diff in-situ matches for each reflectance point so just have one match
import datetime
margin = datetime.timedelta(hours=1) #datetime allowed hour difference
begin_time = datetime.datetime.now()

#Make empty lists which will apped match values to:
temp = []
sal = []
dateandtime =[]


for i in range(0,satDows.shape[0]):       #iterate throough rows of Satellite

    time_diff_list = []
    n_matched = []
    
    
    print(i)
    for n in range(666000,insituDows.shape[0]): # rows of in-situ non-zero start (longer timeseries)
        # this is where matching...
        #insiturow = TempSal.iloc[n,:] 
                  
        #converting string to datetime      
        day = (satDows.iloc[0,3])
        datesat = datetime.datetime.strptime(day, '%Y-%m-%dT%H:%M:%S')

        day2 = (insituDows.iloc[0,1])
        dateinsitu = datetime.datetime.strptime(day2,'%d/%m/%Y %H:%M')

        #Difference intime between in-situ and Reflectance values 
        time_diff = abs(dateinsitu - datesat)
        #If matching location and time save temp, sal etc and RRS values
        if time_diff <= margin :#/
            # code that inputs those matched values into dateframe df
                        
            n_matched.append(n)
            time_diff_list.append(time_diff) #appending matched values to the list
                       
            
    time_diff_array = np.array(time_diff_list)
    n_matched_array = np.array(n_matched)        #making arrays of time_diff next to the row number 

    stacked = np.vstack((n_matched_array,time_diff_array))
    transposed = np.transpose(stacked)
    sortedArr = transposed[transposed[:,1].argsort()]  #sorting the array based on time
    #taking only the smallest 10 time differences. 
    top10rows = sortedArr[0:10,:]

    for m in top10rows[:,0]:        
            
        temp.append(TempSal.iloc[m,8])
        sal.append(TempSal.iloc[m,9])
                    
            
        matchsat = satDows.iloc[i,:].to_numpy()
        SatMatched  = np.vstack((SatMatched,matchsat))

    print('Length of Temp list =', len(temp))
    
    if i == 0: 
        print(datetime.datetime.now() - begin_time)
        
        print((datetime.datetime.now() - begin_time)*len(satDows)) #Estimating length of running code